# 🧪 MULTI-SEED STABILITY TEST
## Kiểm tra độ ổn định của HGT Knowledge Graph Model

**Mục tiêu:** Chạy model với 10 seeds khác nhau để đánh giá:
- Mean AUC
- Standard Deviation
- Confidence Interval
- So sánh với ECFP4 baseline

In [2]:
# =============================================================================
# 2. UTILITY FUNCTIONS
# =============================================================================

def canonicalize_smiles(smi):
    try:
        mol = Chem.MolFromSmiles(smi)
        if mol:
            return Chem.MolToSmiles(mol, canonical=True, isomericSmiles=False)
        return None
    except:
        return None

def scaffold_split_balanced(df, smiles_col='SMILES', test_size=0.2, seed=42):
    """Scaffold split với seed cố định cho reproducibility"""
    np.random.seed(seed)
    scaffold_dict = defaultdict(list)
    
    for idx, smi in enumerate(df[smiles_col]):
        mol = Chem.MolFromSmiles(smi)
        try:
            scaf = MurckoScaffold.MurckoScaffoldSmiles(mol=mol, includeChirality=False) if mol else "Generic"
        except:
            scaf = "Generic"
        scaffold_dict[scaf].append(idx)
    
    groups = sorted(scaffold_dict.values(), key=len, reverse=True)
    train_inds, test_inds = [], []
    target_test_size = len(df) * test_size
    
    for group in groups:
        if len(test_inds) + len(group) <= target_test_size + 50:
            test_inds.extend(group)
        else:
            train_inds.extend(group)
    
    return df.iloc[train_inds].copy(), df.iloc[test_inds].copy()

def get_ecfp4(smiles_list):
    return np.array([np.array(AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(s), 2, nBits=1024)) 
                     for s in smiles_list])

In [ ]:
# =============================================================================
# 4. BUILD TRULY INDEPENDENT GRAPHS (STRICT SEPARATION)
# =============================================================================
# [FIX DATA LEAK] - Complete rewrite to ensure zero overlap

def build_independent_graphs(gds, train_mol_ids, test_mol_ids, seed):
    """
    Build COMPLETELY INDEPENDENT train and test graphs
    
    CRITICAL CHANGES:
    1. Train graph contains ONLY train molecules + their connections
    2. Test graph contains ONLY test molecules + their connections
    3. Shared entities (Scaffold, Target, etc.) use SAME features BUT different local indices
    4. NO topology leakage - graphs are disjoint
    """
    # ==================== SANITY CHECK ====================
    # [FIX DATA LEAK] - Verify no overlap
    overlap = set(train_mol_ids) & set(test_mol_ids)
    assert len(overlap) == 0, f"❌ LEAKAGE DETECTED: {len(overlap)} overlapping molecules!"
    
    # ==================== LOAD ALL ENTITIES ====================
    all_molecules = gds.run_cypher("""
        MATCH (n:Molecule) WHERE n.is_virtual = false
        RETURN id(n) as node_id, n.smiles as smiles
    """)
    
    scaffolds = gds.run_cypher("MATCH (n:Scaffold) RETURN id(n) as node_id, n.smiles as smiles")
    targets = gds.run_cypher("MATCH (n:Target) RETURN id(n) as node_id, n.name as name")
    functional_groups = gds.run_cypher("MATCH (n:FunctionalGroup) RETURN id(n) as node_id, n.name as name")
    warheads = gds.run_cypher("MATCH (n:Warhead) RETURN id(n) as node_id, n.name as name")
    moas = gds.run_cypher("MATCH (n:MoA) RETURN id(n) as node_id, n.name as name")
    
    # ==================== LOAD ALL EDGES ====================
    all_edges = gds.run_cypher("""
        MATCH (m)-[r]->(n)
        WHERE (m:Molecule AND m.is_virtual = false) OR (n:Molecule AND n.is_virtual = false)
        RETURN id(m) as src, type(r) as rel_type, id(n) as dst, 
               labels(m)[0] as src_type, labels(n)[0] as dst_type
    """)
    
    # ==================== SPLIT MOLECULES ====================
    train_molecules = all_molecules[all_molecules['node_id'].isin(train_mol_ids)].reset_index(drop=True)
    test_molecules = all_molecules[all_molecules['node_id'].isin(test_mol_ids)].reset_index(drop=True)
    
    print(f"   ✅ Train: {len(train_molecules)} mols, Test: {len(test_molecules)} mols")
    
    # ==================== INITIALIZE SHARED FEATURES ====================
    # [FIX DATA LEAK] - Features are shared BUT indices are separate
    torch.manual_seed(seed)
    embed_dim = 128
    
    # Global feature matrices (SAME for both graphs)
    global_scaffold_feats = torch.randn(len(scaffolds), embed_dim)
    global_target_feats = torch.randn(len(targets), embed_dim)
    global_fg_feats = torch.randn(len(functional_groups), embed_dim)
    global_wh_feats = torch.randn(len(warheads), embed_dim)
    global_moa_feats = torch.randn(len(moas), embed_dim)
    
    # Global ID to feature index mapping
    global_scaf_map = {row['node_id']: i for i, row in scaffolds.iterrows()}
    global_targ_map = {row['node_id']: i for i, row in targets.iterrows()}
    global_fg_map = {row['node_id']: i for i, row in functional_groups.iterrows()}
    global_wh_map = {row['node_id']: i for i, row in warheads.iterrows()}
    global_moa_map = {row['node_id']: i for i, row in moas.iterrows()}
    
    # ==================== BUILD TRAIN GRAPH ====================
    train_data = HeteroData()
    
    # LOCAL indexing for train molecules
    train_mol_local_map = {row['node_id']: i for i, row in train_molecules.iterrows()}
    
    # Collect entities actually connected to train molecules
    train_connected_entities = {
        'Scaffold': set(), 'Target': set(), 'FunctionalGroup': set(),
        'Warhead': set(), 'MoA': set()
    }
    
    # Filter edges to train graph only
    train_edges = []
    for _, row in all_edges.iterrows():
        src_type, dst_type = row['src_type'], row['dst_type']
        src_id, dst_id = row['src'], row['dst']
        
        # [FIX DATA LEAK] - CRITICAL: Skip ANY edge touching test molecules
        if src_type == 'Molecule' and src_id not in train_mol_local_map:
            continue
        if dst_type == 'Molecule' and dst_id not in train_mol_local_map:
            continue
        
        # Track connected entities
        if src_type != 'Molecule':
            train_connected_entities[src_type].add(src_id)
        if dst_type != 'Molecule':
            train_connected_entities[dst_type].add(dst_id)
        
        train_edges.append(row)
    
    # Create LOCAL mappings for shared entities in train graph
    train_scaf_local = {gid: i for i, gid in enumerate(sorted(train_connected_entities['Scaffold']))}
    train_targ_local = {gid: i for i, gid in enumerate(sorted(train_connected_entities['Target']))}
    train_fg_local = {gid: i for i, gid in enumerate(sorted(train_connected_entities['FunctionalGroup']))}
    train_wh_local = {gid: i for i, gid in enumerate(sorted(train_connected_entities['Warhead']))}
    train_moa_local = {gid: i for i, gid in enumerate(sorted(train_connected_entities['MoA']))}
    
    # Set features (select from global features)
    train_data['Molecule'].x = torch.randn(len(train_molecules), embed_dim)
    train_data['Scaffold'].x = global_scaffold_feats[[global_scaf_map[gid] for gid in sorted(train_connected_entities['Scaffold'])]]
    train_data['Target'].x = global_target_feats[[global_targ_map[gid] for gid in sorted(train_connected_entities['Target'])]]
    train_data['FunctionalGroup'].x = global_fg_feats[[global_fg_map[gid] for gid in sorted(train_connected_entities['FunctionalGroup'])]]
    train_data['Warhead'].x = global_wh_feats[[global_wh_map[gid] for gid in sorted(train_connected_entities['Warhead'])]]
    train_data['MoA'].x = global_moa_feats[[global_moa_map[gid] for gid in sorted(train_connected_entities['MoA'])]]
    
    # Build train edges with local indices
    train_edge_dict = defaultdict(lambda: [[], []])
    train_local_maps = {
        'Molecule': train_mol_local_map,
        'Scaffold': train_scaf_local,
        'Target': train_targ_local,
        'FunctionalGroup': train_fg_local,
        'Warhead': train_wh_local,
        'MoA': train_moa_local
    }
    
    for row in train_edges:
        src_type, dst_type = row['src_type'], row['dst_type']
        src_id, dst_id = row['src'], row['dst']
        
        if src_id not in train_local_maps[src_type] or dst_id not in train_local_maps[dst_type]:
            continue
        
        src_idx = train_local_maps[src_type][src_id]
        dst_idx = train_local_maps[dst_type][dst_id]
        
        key = (src_type, row['rel_type'], dst_type)
        train_edge_dict[key][0].append(src_idx)
        train_edge_dict[key][1].append(dst_idx)
        
        # Reverse edges
        rev_key = (dst_type, f"rev_{row['rel_type']}", src_type)
        train_edge_dict[rev_key][0].append(dst_idx)
        train_edge_dict[rev_key][1].append(src_idx)
    
    for key, (srcs, dsts) in train_edge_dict.items():
        train_data[key].edge_index = torch.tensor([srcs, dsts], dtype=torch.long)
    
    # ==================== BUILD TEST GRAPH (SAME PROCESS) ====================
    test_data = HeteroData()
    
    test_mol_local_map = {row['node_id']: i for i, row in test_molecules.iterrows()}
    
    test_connected_entities = {
        'Scaffold': set(), 'Target': set(), 'FunctionalGroup': set(),
        'Warhead': set(), 'MoA': set()
    }
    
    test_edges = []
    for _, row in all_edges.iterrows():
        src_type, dst_type = row['src_type'], row['dst_type']
        src_id, dst_id = row['src'], row['dst']
        
        # [FIX DATA LEAK] - CRITICAL: Skip ANY edge touching train molecules
        if src_type == 'Molecule' and src_id not in test_mol_local_map:
            continue
        if dst_type == 'Molecule' and dst_id not in test_mol_local_map:
            continue
        
        if src_type != 'Molecule':
            test_connected_entities[src_type].add(src_id)
        if dst_type != 'Molecule':
            test_connected_entities[dst_type].add(dst_id)
        
        test_edges.append(row)
    
    test_scaf_local = {gid: i for i, gid in enumerate(sorted(test_connected_entities['Scaffold']))}
    test_targ_local = {gid: i for i, gid in enumerate(sorted(test_connected_entities['Target']))}
    test_fg_local = {gid: i for i, gid in enumerate(sorted(test_connected_entities['FunctionalGroup']))}
    test_wh_local = {gid: i for i, gid in enumerate(sorted(test_connected_entities['Warhead']))}
    test_moa_local = {gid: i for i, gid in enumerate(sorted(test_connected_entities['MoA']))}
    
    test_data['Molecule'].x = torch.randn(len(test_molecules), embed_dim)
    test_data['Scaffold'].x = global_scaffold_feats[[global_scaf_map[gid] for gid in sorted(test_connected_entities['Scaffold'])]]
    test_data['Target'].x = global_target_feats[[global_targ_map[gid] for gid in sorted(test_connected_entities['Target'])]]
    test_data['FunctionalGroup'].x = global_fg_feats[[global_fg_map[gid] for gid in sorted(test_connected_entities['FunctionalGroup'])]]
    test_data['Warhead'].x = global_wh_feats[[global_wh_map[gid] for gid in sorted(test_connected_entities['Warhead'])]]
    test_data['MoA'].x = global_moa_feats[[global_moa_map[gid] for gid in sorted(test_connected_entities['MoA'])]]
    
    test_edge_dict = defaultdict(lambda: [[], []])
    test_local_maps = {
        'Molecule': test_mol_local_map,
        'Scaffold': test_scaf_local,
        'Target': test_targ_local,
        'FunctionalGroup': test_fg_local,
        'Warhead': test_wh_local,
        'MoA': test_moa_local
    }
    
    for row in test_edges:
        src_type, dst_type = row['src_type'], row['dst_type']
        src_id, dst_id = row['src'], row['dst']
        
        if src_id not in test_local_maps[src_type] or dst_id not in test_local_maps[dst_type]:
            continue
        
        src_idx = test_local_maps[src_type][src_id]
        dst_idx = test_local_maps[dst_type][dst_id]
        
        key = (src_type, row['rel_type'], dst_type)
        test_edge_dict[key][0].append(src_idx)
        test_edge_dict[key][1].append(dst_idx)
        
        rev_key = (dst_type, f"rev_{row['rel_type']}", src_type)
        test_edge_dict[rev_key][0].append(dst_idx)
        test_edge_dict[rev_key][1].append(src_idx)
    
    for key, (srcs, dsts) in test_edge_dict.items():
        test_data[key].edge_index = torch.tensor([srcs, dsts], dtype=torch.long)
    
    print(f"   ✅ Train edges: {sum(len(e[0]) for e in train_edge_dict.values())}, Test edges: {sum(len(e[0]) for e in test_edge_dict.values())}")
    
    return train_data, test_data, train_mol_local_map, test_mol_local_map, train_molecules, test_molecules

In [4]:
# =============================================================================
# 4. BUILD TRULY INDEPENDENT GRAPHS (STRICT SEPARATION)
# =============================================================================
# [FIX DATA LEAK] - Complete rewrite to ensure zero overlap

def build_independent_graphs(gds, train_mol_ids, test_mol_ids, seed):
    """
    Build COMPLETELY INDEPENDENT train and test graphs
    
    CRITICAL CHANGES:
    1. Train graph contains ONLY train molecules + their connections
    2. Test graph contains ONLY test molecules + their connections
    3. Shared entities (Scaffold, Target, etc.) use SAME features BUT different local indices
    4. NO topology leakage - graphs are disjoint
    """
    # ==================== SANITY CHECK ====================
    # [FIX DATA LEAK] - Verify no overlap
    overlap = set(train_mol_ids) & set(test_mol_ids)
    assert len(overlap) == 0, f"❌ LEAKAGE DETECTED: {len(overlap)} overlapping molecules!"
    
    # ==================== LOAD ALL ENTITIES ====================
    all_molecules = gds.run_cypher("""
        MATCH (n:Molecule) WHERE n.is_virtual = false
        RETURN id(n) as node_id, n.smiles as smiles
    """)
    
    scaffolds = gds.run_cypher("MATCH (n:Scaffold) RETURN id(n) as node_id, n.smiles as smiles")
    targets = gds.run_cypher("MATCH (n:Target) RETURN id(n) as node_id, n.name as name")
    functional_groups = gds.run_cypher("MATCH (n:FunctionalGroup) RETURN id(n) as node_id, n.name as name")
    warheads = gds.run_cypher("MATCH (n:Warhead) RETURN id(n) as node_id, n.name as name")
    moas = gds.run_cypher("MATCH (n:MoA) RETURN id(n) as node_id, n.name as name")
    
    # ==================== LOAD ALL EDGES ====================
    all_edges = gds.run_cypher("""
        MATCH (m)-[r]->(n)
        WHERE (m:Molecule AND m.is_virtual = false) OR (n:Molecule AND n.is_virtual = false)
        RETURN id(m) as src, type(r) as rel_type, id(n) as dst, 
               labels(m)[0] as src_type, labels(n)[0] as dst_type
    """)
    
    # ==================== SPLIT MOLECULES ====================
    train_molecules = all_molecules[all_molecules['node_id'].isin(train_mol_ids)].reset_index(drop=True)
    test_molecules = all_molecules[all_molecules['node_id'].isin(test_mol_ids)].reset_index(drop=True)
    
    print(f"   ✅ Train: {len(train_molecules)} mols, Test: {len(test_molecules)} mols")
    
    # ==================== INITIALIZE SHARED FEATURES ====================
    # [FIX DATA LEAK] - Features are shared BUT indices are separate
    torch.manual_seed(seed)
    embed_dim = 128
    
    # Global feature matrices (SAME for both graphs)
    global_scaffold_feats = torch.randn(len(scaffolds), embed_dim)
    global_target_feats = torch.randn(len(targets), embed_dim)
    global_fg_feats = torch.randn(len(functional_groups), embed_dim)
    global_wh_feats = torch.randn(len(warheads), embed_dim)
    global_moa_feats = torch.randn(len(moas), embed_dim)
    
    # Global ID to feature index mapping
    global_scaf_map = {row['node_id']: i for i, row in scaffolds.iterrows()}
    global_targ_map = {row['node_id']: i for i, row in targets.iterrows()}
    global_fg_map = {row['node_id']: i for i, row in functional_groups.iterrows()}
    global_wh_map = {row['node_id']: i for i, row in warheads.iterrows()}
    global_moa_map = {row['node_id']: i for i, row in moas.iterrows()}
    
    # ==================== BUILD TRAIN GRAPH ====================
    train_data = HeteroData()
    
    # LOCAL indexing for train molecules
    train_mol_local_map = {row['node_id']: i for i, row in train_molecules.iterrows()}
    
    # Collect entities actually connected to train molecules
    train_connected_entities = {
        'Scaffold': set(), 'Target': set(), 'FunctionalGroup': set(),
        'Warhead': set(), 'MoA': set()
    }
    
    # Filter edges to train graph only
    train_edges = []
    for _, row in all_edges.iterrows():
        src_type, dst_type = row['src_type'], row['dst_type']
        src_id, dst_id = row['src'], row['dst']
        
        # [FIX DATA LEAK] - CRITICAL: Skip ANY edge touching test molecules
        if src_type == 'Molecule' and src_id not in train_mol_local_map:
            continue
        if dst_type == 'Molecule' and dst_id not in train_mol_local_map:
            continue
        
        # Track connected entities
        if src_type != 'Molecule':
            train_connected_entities[src_type].add(src_id)
        if dst_type != 'Molecule':
            train_connected_entities[dst_type].add(dst_id)
        
        train_edges.append(row)
    
    # Create LOCAL mappings for shared entities in train graph
    train_scaf_local = {gid: i for i, gid in enumerate(sorted(train_connected_entities['Scaffold']))}
    train_targ_local = {gid: i for i, gid in enumerate(sorted(train_connected_entities['Target']))}
    train_fg_local = {gid: i for i, gid in enumerate(sorted(train_connected_entities['FunctionalGroup']))}
    train_wh_local = {gid: i for i, gid in enumerate(sorted(train_connected_entities['Warhead']))}
    train_moa_local = {gid: i for i, gid in enumerate(sorted(train_connected_entities['MoA']))}
    
    # Set features (select from global features)
    train_data['Molecule'].x = torch.randn(len(train_molecules), embed_dim)
    train_data['Scaffold'].x = global_scaffold_feats[[global_scaf_map[gid] for gid in sorted(train_connected_entities['Scaffold'])]]
    train_data['Target'].x = global_target_feats[[global_targ_map[gid] for gid in sorted(train_connected_entities['Target'])]]
    train_data['FunctionalGroup'].x = global_fg_feats[[global_fg_map[gid] for gid in sorted(train_connected_entities['FunctionalGroup'])]]
    train_data['Warhead'].x = global_wh_feats[[global_wh_map[gid] for gid in sorted(train_connected_entities['Warhead'])]]
    train_data['MoA'].x = global_moa_feats[[global_moa_map[gid] for gid in sorted(train_connected_entities['MoA'])]]
    
    # Build train edges with local indices
    train_edge_dict = defaultdict(lambda: [[], []])
    train_local_maps = {
        'Molecule': train_mol_local_map,
        'Scaffold': train_scaf_local,
        'Target': train_targ_local,
        'FunctionalGroup': train_fg_local,
        'Warhead': train_wh_local,
        'MoA': train_moa_local
    }
    
    for row in train_edges:
        src_type, dst_type = row['src_type'], row['dst_type']
        src_id, dst_id = row['src'], row['dst']
        
        if src_id not in train_local_maps[src_type] or dst_id not in train_local_maps[dst_type]:
            continue
        
        src_idx = train_local_maps[src_type][src_id]
        dst_idx = train_local_maps[dst_type][dst_id]
        
        key = (src_type, row['rel_type'], dst_type)
        train_edge_dict[key][0].append(src_idx)
        train_edge_dict[key][1].append(dst_idx)
        
        # Reverse edges
        rev_key = (dst_type, f"rev_{row['rel_type']}", src_type)
        train_edge_dict[rev_key][0].append(dst_idx)
        train_edge_dict[rev_key][1].append(src_idx)
    
    for key, (srcs, dsts) in train_edge_dict.items():
        train_data[key].edge_index = torch.tensor([srcs, dsts], dtype=torch.long)
    
    # ==================== BUILD TEST GRAPH (SAME PROCESS) ====================
    test_data = HeteroData()
    
    test_mol_local_map = {row['node_id']: i for i, row in test_molecules.iterrows()}
    
    test_connected_entities = {
        'Scaffold': set(), 'Target': set(), 'FunctionalGroup': set(),
        'Warhead': set(), 'MoA': set()
    }
    
    test_edges = []
    for _, row in all_edges.iterrows():
        src_type, dst_type = row['src_type'], row['dst_type']
        src_id, dst_id = row['src'], row['dst']
        
        # [FIX DATA LEAK] - CRITICAL: Skip ANY edge touching train molecules
        if src_type == 'Molecule' and src_id not in test_mol_local_map:
            continue
        if dst_type == 'Molecule' and dst_id not in test_mol_local_map:
            continue
        
        if src_type != 'Molecule':
            test_connected_entities[src_type].add(src_id)
        if dst_type != 'Molecule':
            test_connected_entities[dst_type].add(dst_id)
        
        test_edges.append(row)
    
    test_scaf_local = {gid: i for i, gid in enumerate(sorted(test_connected_entities['Scaffold']))}
    test_targ_local = {gid: i for i, gid in enumerate(sorted(test_connected_entities['Target']))}
    test_fg_local = {gid: i for i, gid in enumerate(sorted(test_connected_entities['FunctionalGroup']))}
    test_wh_local = {gid: i for i, gid in enumerate(sorted(test_connected_entities['Warhead']))}
    test_moa_local = {gid: i for i, gid in enumerate(sorted(test_connected_entities['MoA']))}
    
    test_data['Molecule'].x = torch.randn(len(test_molecules), embed_dim)
    test_data['Scaffold'].x = global_scaffold_feats[[global_scaf_map[gid] for gid in sorted(test_connected_entities['Scaffold'])]]
    test_data['Target'].x = global_target_feats[[global_targ_map[gid] for gid in sorted(test_connected_entities['Target'])]]
    test_data['FunctionalGroup'].x = global_fg_feats[[global_fg_map[gid] for gid in sorted(test_connected_entities['FunctionalGroup'])]]
    test_data['Warhead'].x = global_wh_feats[[global_wh_map[gid] for gid in sorted(test_connected_entities['Warhead'])]]
    test_data['MoA'].x = global_moa_feats[[global_moa_map[gid] for gid in sorted(test_connected_entities['MoA'])]]
    
    test_edge_dict = defaultdict(lambda: [[], []])
    test_local_maps = {
        'Molecule': test_mol_local_map,
        'Scaffold': test_scaf_local,
        'Target': test_targ_local,
        'FunctionalGroup': test_fg_local,
        'Warhead': test_wh_local,
        'MoA': test_moa_local
    }
    
    for row in test_edges:
        src_type, dst_type = row['src_type'], row['dst_type']
        src_id, dst_id = row['src'], row['dst']
        
        if src_id not in test_local_maps[src_type] or dst_id not in test_local_maps[dst_type]:
            continue
        
        src_idx = test_local_maps[src_type][src_id]
        dst_idx = test_local_maps[dst_type][dst_id]
        
        key = (src_type, row['rel_type'], dst_type)
        test_edge_dict[key][0].append(src_idx)
        test_edge_dict[key][1].append(dst_idx)
        
        rev_key = (dst_type, f"rev_{row['rel_type']}", src_type)
        test_edge_dict[rev_key][0].append(dst_idx)
        test_edge_dict[rev_key][1].append(src_idx)
    
    for key, (srcs, dsts) in test_edge_dict.items():
        test_data[key].edge_index = torch.tensor([srcs, dsts], dtype=torch.long)
    
    print(f"   ✅ Train edges: {sum(len(e[0]) for e in train_edge_dict.values())}, Test edges: {sum(len(e[0]) for e in test_edge_dict.values())}")
    
    return train_data, test_data, train_mol_local_map, test_mol_local_map, train_molecules, test_molecules

In [6]:
# =============================================================================
# 6. RUN MULTI-SEED EXPERIMENTS
# =============================================================================
# [FIX DATA LEAK] - Keep same, but now calls corrected pipeline

# Load data once
df = pd.read_csv(CSV_PATH)
df = df[df['Final_Label'] != 'Uncertain']
df['Label_Num'] = df['Final_Label'].apply(lambda x: 1 if str(x).lower() == 'active' else 0)
df['SMILES_Key'] = df['SMILES'].apply(canonicalize_smiles)
df = df.dropna(subset=['SMILES_Key']).drop_duplicates(subset=['SMILES_Key'])

gds = GraphDataScience(NEO4J_URI, auth=NEO4J_AUTH)

print("\n" + "="*80)
print("🚀 MULTI-SEED STABILITY TEST (CORRECTED - NO LEAKAGE)")
print(f"   Testing {len(TEST_SEEDS)} seeds")
print("="*80)
print("\n✅ FIXES APPLIED:")
print("   1. Scaffold split BEFORE graph construction")
print("   2. Completely independent train/test graphs")
print("   3. Self-supervised pretraining (NO label leakage)")
print("   4. FROZEN encoder for test inference")
print("   5. Sanity checks (permutation + random baseline)")
print("="*80)

# Run experiments
results = []
for i, seed in enumerate(TEST_SEEDS, 1):
    print(f"\n[{i}/{len(TEST_SEEDS)}] Seed = {seed}")
    result = run_single_experiment(seed, gds, df, verbose=True)
    results.append(result)

results_df = pd.DataFrame(results)
print("\n✅ All experiments completed!")


🚀 MULTI-SEED STABILITY TEST (CORRECTED - NO LEAKAGE)
   Testing 10 seeds

✅ FIXES APPLIED:
   1. Scaffold split BEFORE graph construction
   2. Completely independent train/test graphs
   3. Self-supervised pretraining (NO label leakage)
   4. FROZEN encoder for test inference
   5. Sanity checks (permutation + random baseline)

[1/10] Seed = 42

🎲 Seed = 42
  🔀 Split: Train=4478, Test=1166
   ✅ Train: 4478 mols, Test: 1166 mols
   ✅ Train edges: 164694, Test edges: 41818
    Epoch 10/50 | Self-Supervised Loss: 1.3851
    Epoch 20/50 | Self-Supervised Loss: 1.3833
    Epoch 30/50 | Self-Supervised Loss: 1.3789
    Epoch 40/50 | Self-Supervised Loss: 1.3732
    Epoch 50/50 | Self-Supervised Loss: 1.3662
  📊 ECFP4       -> AUC: 0.9575, ACC: 0.8808
  📊 HGT (SS)    -> AUC: 0.9164, ACC: 0.8405
  🔍 Sanity: Permuted=0.5400, Random=0.4940
  ✅ Graphs are independent (NO leakage)

[2/10] Seed = 123

🎲 Seed = 123
  🔀 Split: Train=4478, Test=1166
   ✅ Train: 4478 mols, Test: 1166 mols
   ✅ Train 

## Kịch bản thuyết trình (Slide: Stability Analysis – Multi-seed)

**(Mở đầu – 10–15s)**
- Dạ em xin trình bày slide đánh giá **độ ổn định theo nhiều seed** của mô hình HGT trên Knowledge Graph, và so sánh với baseline **ECFP4**.
- Mục tiêu của phần này là kiểm tra: khi thay đổi seed (khởi tạo/ngẫu nhiên), hiệu năng AUC-ROC **dao động nhiều hay ít**.

**(Bối cảnh thí nghiệm – 20–30s)**
- Em chạy cùng một pipeline và cùng cách chia dữ liệu (scaffold split) nhưng thay **10 seeds** khác nhau.
- Mỗi seed cho ra một giá trị **AUC-ROC** cho 2 phương pháp: **ECFP4** và **KG/HGT**.
- Ở đây em tập trung vào hai ý: (1) mức AUC trung bình và (2) **độ ổn định** thể hiện qua phân phối và độ lệch chuẩn.

**(Giải thích biểu đồ – đi theo 4 ô, 60–90s)**
1) **Ô trên-trái: Violin plot – AUC Distribution Across Seeds**
- Biểu đồ này cho thấy **phân phối AUC** theo 10 seed cho từng phương pháp.
- Phần violin càng “dày” ở đâu thì AUC hay rơi vào vùng đó; đồng thời nhìn được độ trải rộng.
- Em nhấn mạnh: ECFP4 có phân phối **rất hẹp** → ổn định; còn KG có phân phối **rộng hơn** → nhạy hơn với seed.

2) **Ô trên-phải: AUC Across Different Seeds (theo từng seed)**
- Mỗi điểm là một seed (S1…S10).
- Đường ECFP4 gần như phẳng → thay seed gần như không đổi.
- Đường KG dao động rõ hơn và có seed tụt mạnh hơn các seed còn lại → cho thấy rủi ro “hên xui” theo khởi tạo hoặc quá trình học.

3) **Ô dưới-trái: Box plot + điểm từng seed (AUC Stability)**
- Box plot tóm tắt trung vị và khoảng tứ phân vị; các điểm đỏ/xanh là kết quả từng seed.
- ECFP4: các điểm tụ lại, hộp rất gọn.
- KG: hộp và whisker rộng hơn, có điểm lệch nhiều → thể hiện variance cao hơn.

4) **Ô dưới-phải: Standard Deviation (Lower is better)**
- Đây là tóm tắt định lượng: **độ lệch chuẩn AUC** của mỗi phương pháp.
- ECFP4 có std rất thấp (trong slide đang hiển thị khoảng ~0.0017), còn KG cao hơn đáng kể (khoảng ~0.0271).
- Kết luận từ ô này: xét riêng về **stability**, ECFP4 tốt hơn; KG cần cải thiện để kết quả nhất quán hơn.

**(Kết luận & thông điệp chốt – 20–30s)**
- Slide này trả lời câu hỏi: “Mô hình có ổn định không khi đổi seed?”
- Kết quả: **ECFP4 ổn định**, còn **KG/HGT có thể đạt AUC tốt** nhưng **dao động theo seed** tương đối lớn.
- Hướng cải thiện em đề xuất: cố định/kiểm soát nguồn ngẫu nhiên kỹ hơn, tinh chỉnh hyperparameter, regularization/early stopping ổn định hơn, và báo cáo thêm mean±CI trên nhiều seed để đánh giá công bằng.

In [1]:
# =============================================================================
# 8. VISUALIZATION (NO CHANGES)
# =============================================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# --- Plot 1: AUC Distribution (Violin Plot) ---
ax1 = axes[0, 0]
data_plot = pd.melt(results_df[['auc_ecfp', 'auc_kg']], 
                     var_name='Method', value_name='AUC')
data_plot['Method'] = data_plot['Method'].map({'auc_ecfp': 'ECFP4', 'auc_kg': 'KG'})
sns.violinplot(data=data_plot, x='Method', y='AUC', ax=ax1, palette=['skyblue', 'salmon'])
ax1.set_title('AUC Distribution Across Seeds', fontsize=14, fontweight='bold')
ax1.set_ylabel('AUC-ROC', fontsize=12)
ax1.grid(axis='y', alpha=0.3)

# --- Plot 2: Seed-by-Seed Comparison ---
ax2 = axes[0, 1]
x = np.arange(len(results_df))
ax2.plot(x, results_df['auc_ecfp'], marker='o', label='ECFP4', color='blue', linewidth=2)
ax2.plot(x, results_df['auc_kg'], marker='s', label='KG', color='red', linewidth=2)
ax2.fill_between(x, results_df['auc_ecfp'], alpha=0.2, color='blue')
ax2.fill_between(x, results_df['auc_kg'], alpha=0.2, color='red')
ax2.set_xticks(x)
ax2.set_xticklabels([f"S{i+1}" for i in range(len(results_df))], rotation=45)
ax2.set_xlabel('Seed Index', fontsize=12)
ax2.set_ylabel('AUC-ROC', fontsize=12)
ax2.set_title('AUC Across Different Seeds', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)

# --- Plot 3: Box Plot with Individual Points ---
ax3 = axes[1, 0]
bp = ax3.boxplot([results_df['auc_ecfp'], results_df['auc_kg']], 
                   labels=['ECFP4', 'KG'], patch_artist=True,
                   boxprops=dict(facecolor='lightgray', alpha=0.7))
for i, (method, color) in enumerate([('auc_ecfp', 'blue'), ('auc_kg', 'red')], 1):
    y = results_df[method]
    x = np.random.normal(i, 0.04, size=len(y))
    ax3.scatter(x, y, alpha=0.6, color=color, s=50, zorder=3)
ax3.set_ylabel('AUC-ROC', fontsize=12)
ax3.set_title('AUC Stability (Box Plot)', fontsize=14, fontweight='bold')
ax3.grid(axis='y', alpha=0.3)

# --- Plot 4: Standard Deviation Comparison ---
ax4 = axes[1, 1]
methods = ['ECFP4', 'KG']
stds = [results_df['auc_ecfp'].std(), results_df['auc_kg'].std()]
colors = ['skyblue', 'salmon']
bars = ax4.bar(methods, stds, color=colors, edgecolor='black', linewidth=1.5)
ax4.set_ylabel('Standard Deviation', fontsize=12)
ax4.set_title('Stability Comparison (Lower is Better)', fontsize=14, fontweight='bold')
ax4.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar, std in zip(bars, stds):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height,
             f'{std:.4f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('stability_analysis_corrected.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n💾 Plot saved as 'stability_analysis_corrected.png'")

NameError: name 'plt' is not defined